# Compute KS-test results per feature between the two different dervivatives of the `ipn02.3 2λ` cell line

Plate 6 contains two derivatives of the cell line acquired from `iNFixion` and `MGH`.

## Import libraries

In [1]:
import pathlib
import pandas as pd
from scipy import stats
from joblib import load

## Set results directory and load in model to get list of the features used

In [2]:
# Set results directory
results_dir = pathlib.Path("./results")
results_dir.mkdir(exist_ok=True)

# Set data type for the ks test evaluation
data_type = "cleaned"

# Set suffix for data files if using QC or cleaned data
if data_type == "cleaned":
    suffix = "_qc"
else:
    suffix = ""

# Load in model
model = load(
    pathlib.Path(f"../1.train_models/data/trained_nf1_model{suffix}.joblib").resolve(
        strict=True
    )
)
model_features = list(model.feature_names_in_)

len(model_features)

894

## Load in Plate 6 normalized data

In [3]:
# Set directory to find the plate 6 data from based on data type
directory = (
    "single_cell_profiles/cleaned_sc_profiles"
    if data_type == "cleaned"
    else "single_cell_profiles"
)

# Load in the normalized data
plate_6_norm = pd.read_parquet(
    pathlib.Path(
        f"/media/18tbdrive/1.Github_Repositories/nf1_schwann_cell_painting_data/3.processing_features/data/{directory}/Plate_6_sc_normalized.parquet"
    )
)

## Perform KS-test comparing the features between the two cell line derivatives

In [4]:
# Split data by institution for comparison
institution_1_norm = plate_6_norm[plate_6_norm["Metadata_Institution"] == "iNFixion"]
institution_2_norm = plate_6_norm[plate_6_norm["Metadata_Institution"] == "MGH"]

# Perform KS-test for each feature
ks_test_results_norm = {}

for column in plate_6_norm.columns:
    if column.startswith("Metadata_"):
        continue
    ks_stat, p_value = stats.kstest(
        institution_1_norm[column], institution_2_norm[column]
    )
    ks_test_results_norm[column] = {"ks_stat": ks_stat, "p_value": p_value}

# Convert results to DataFrame for better visualization
ks_test_results_norm_df = (
    pd.DataFrame(ks_test_results_norm)
    .T.reset_index()
    .rename(columns={"index": "feature"})
)

In [5]:
print("\nKS-test results for normalized data:")
ks_test_results_norm_df.head()


KS-test results for normalized data:


,feature,ks_stat,p_value
0,Cytoplasm_AreaShape_Area,0.712750,2.519735e-322
1,Cytoplasm_AreaShape_BoundingBoxArea,0.629302,1.867568e-321
2,Cytoplasm_AreaShape_BoundingBoxMaximum_X,0.087586,1.416552e-10
3,Cytoplasm_AreaShape_BoundingBoxMaximum_Y,0.112275,4.164913e-17
4,Cytoplasm_AreaShape_BoundingBoxMinimum_X,0.097974,3.988825e-13


## Add absolute value coefficients per feature to the results

In [6]:
feat_import_df = pd.read_parquet(
    pathlib.Path(
        "../2.evaluate_model/model_evaluation_data/feature_importances.parquet"
    )
)

# Take the absolute value of the feature importance
feat_import_df["feature_importances"] = feat_import_df["feature_importances"].abs()

# Change the column name from feature_names to feature
feat_import_df = feat_import_df.rename(columns={"feature_names": "feature"})

# Merge the feature importance data with the KS test results
ks_test_results_norm_df = ks_test_results_norm_df.merge(feat_import_df, on="feature")

ks_test_results_norm_df.head()

,feature,ks_stat,p_value,feature_importances
0,Cytoplasm_AreaShape_Eccentricity,0.054304,2.483386e-04,0.020474
1,Cytoplasm_AreaShape_FormFactor,0.300949,8.887027e-122,1.138960
2,Cytoplasm_AreaShape_MajorAxisLength,0.485603,4.540463e-321,0.169308
3,Cytoplasm_AreaShape_MinorAxisLength,0.552350,3.320121e-321,0.014426
4,Cytoplasm_AreaShape_Orientation,0.017507,7.324997e-01,0.000416


## Split feature names into parts and save results

In [7]:
# Split the feature column into parts
ks_test_results_norm_df[
    [
        "compartment",
        "feature_group",
        "measurement",
        "channel",
        "parameter1",
        "parameter2",
        "parameter3",
    ]
] = (
    ks_test_results_norm_df["feature"]
    .str.split("_", expand=True)
    .reindex(columns=range(7), fill_value=pd.NA)
)

# Filter out features not in model_features
ks_test_results_norm_df = ks_test_results_norm_df[
    ks_test_results_norm_df["feature"].isin(model_features)
]

# Save the results with qc suffix if data is cleaned
if data_type == "cleaned":
    ks_test_results_file = (
        pathlib.Path(results_dir) / "ks_test_derivatives_results_qc.parquet"
    )
else:
    ks_test_results_file = (
        pathlib.Path(results_dir) / "ks_test_derivatives_results.parquet"
    )

ks_test_results_norm_df.to_parquet(ks_test_results_file)

# Display the updated DataFrame
print(ks_test_results_norm_df.shape)
ks_test_results_norm_df.head()

(832, 11)


,feature,ks_stat,p_value,feature_importances,compartment,feature_group,measurement,channel,parameter1,parameter2,parameter3
0,Cytoplasm_AreaShape_Eccentricity,0.054304,2.483386e-04,0.020474,Cytoplasm,AreaShape,Eccentricity,None,None,None,None
1,Cytoplasm_AreaShape_FormFactor,0.300949,8.887027e-122,1.138960,Cytoplasm,AreaShape,FormFactor,None,None,None,None
2,Cytoplasm_AreaShape_MajorAxisLength,0.485603,4.540463e-321,0.169308,Cytoplasm,AreaShape,MajorAxisLength,None,None,None,None
3,Cytoplasm_AreaShape_MinorAxisLength,0.552350,3.320121e-321,0.014426,Cytoplasm,AreaShape,MinorAxisLength,None,None,None,None
5,Cytoplasm_AreaShape_Solidity,0.206567,3.973060e-57,0.216575,Cytoplasm,AreaShape,Solidity,None,None,None,None


## Print rows from the top five feature importances

In [8]:
ks_test_results_norm_df = ks_test_results_norm_df.sort_values(
    by="feature_importances", ascending=False
)
ks_test_results_norm_df.head()

,feature,ks_stat,p_value,feature_importances,compartment,feature_group,measurement,channel,parameter1,parameter2,parameter3
64,Cytoplasm_RadialDistribution_FracAtD_RFP_4of4,0.050312,8.850725e-04,2.338295,Cytoplasm,RadialDistribution,FracAtD,RFP,4of4,None,None
291,Cells_Correlation_Correlation_DAPI_GFP,0.081385,3.442998e-09,1.991228,Cells,Correlation,Correlation,DAPI,GFP,None,None
495,Cells_Texture_Correlation_DAPI_3_02_256,0.452904,2.712266e-281,1.729736,Cells,Texture,Correlation,DAPI,3,02,256
60,Cytoplasm_RadialDistribution_FracAtD_DAPI_4of4,0.086449,2.587087e-10,1.661875,Cytoplasm,RadialDistribution,FracAtD,DAPI,4of4,None,None
262,Cytoplasm_Texture_InfoMeas1_GFP_3_00_256,0.151330,9.132412e-31,1.441244,Cytoplasm,Texture,InfoMeas1,GFP,3,00,256
